# 1. Preprocessing 

## 1.1 Requirements

In [1]:
pip install pandas

In [2]:
pip install --upgrade nltk

## 1.2 Imports

In [3]:
import pandas as pd
import re
import string
import nltk
nltk.download('stopwords')
#import stanza
#stanza.download('pt')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
#from gensim.models import KeyedVectors
#import torch

[nltk_data] Downloading package stopwords to /home/alina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 1.3 Get data

In [4]:
# Table only with news 'headline' and 'content'

pd.set_option('max_colwidth',150)
data = pd.read_csv ('data_sample.csv')
data_text = data.loc[:, ['headline', 'content']]

In [5]:
# Search for null values

for col in data_text.columns:
    print(col, data_text[col].isnull().sum())

headline 0
content 0


In [6]:
# Join 'headline' and 'content'

for index, row in data_text.iterrows():
    if isinstance(row['headline'], float): pass
    else: 
        if row['headline'] not in row['content']: row['content'] = row['headline'] + ' ' + row['content']
data_text.drop('headline', axis = 1, inplace = True)
data_text

,content
0,"Prisão perpétua para homem que tentou assassinar senadora No dia 8 de Janeiro de 2011 Jared Loughner tentou matar, sem sucesso, a então senadora G..."
1,"John Nash, matemático de Uma Mente Brilhante, morre em acidente John Nash, o matemático e Nobel da Economia que foi retratado por Russell Crowe no..."
2,O mito da reeleição mínima garantida Cavaco sairá desta campanha pior do que entrou. Os casos das acções na SLN ou da casa de férias no Algarve nã...
3,"Morreu Rita Levi-Montalcini, a grande dama da ciência italiana A cientista e senadora italiana Rita Levi-Montalcini, Prémio Nobel da Medicina de 1..."
4,"Por trás da porta amarela, há um homem com problemas psicológicos Pub Ninguém sabe o que ele está a fazer. Ninguém sabe em que é que pensa, o que..."
5,"Gregorio Duvivier de todas as maneiras e sem estilo Outubro será o mês de Gregorio Duvivier em Portugal. O escritor, humorista e actor-sócio fund..."
6,"Ângelo de Sousa: Viver, esperar, talvez pintar Ângelo é nome de curioso. Um que se farta de esperar. Um que procura saber. Conversa no Porto, em c..."
7,"""Sintomas de esquizofrenia"" Fernando Rosas zurze em Ferreira do Amaral ""Sintomas de Esquizofrenia"" Quinta-feira, 11 de Janeiro de 2001 A port..."
8,"Esquizofrenia e depressão Esquizofrenia e Depressão Domingo, 31 de Dezembro de 2000 Ao longo dos seus 117 anos de existência, o Hospital do Cond..."


## 1.4 Process data

In [7]:
# Cleaning

irrelevant = ["pub", "pág", "reportagem", "clique", "subscrever", "partilhar", "dn", "publico.pt", "rtp", "cartaz", "tv", "hoje", "amanhã", "ontem", "segundafeira", "terçafeira", "quartafeira", "quintafeira", "sextafeira", "sábado", "domingo", "janeiro", "fevereiro", "março", "abril", "maio", "junho", "julho", "agosto", "outubro", "novembro", "dezembro"] # list with irrelevant words/expressions do the news content

stopwords = nltk.corpus.stopwords.words('portuguese') # pt stopwords

sent_tokenizer = nltk.data.load('tokenizers/punkt/portuguese.pickle') # pt sentence tokenizer

#nlp = stanza.Pipeline('pt') # pt stanza

def clean_text(text):

    text = text.lower() # lowercase
    text = re.sub(r'http\S+', '', text) # URLs removal
    text = re.sub('\[.*?\]', '', text) # remove text in square brackets
    text = re.sub(r"\([^()]*\)", '', text) # remove text within parentheses
    text = re.sub(r'-(me|te|se|nos|vos|o|os|a|as|lo|los|la|las|lhe|lhes|lha|lhas|lho|lhos|no|na|nas|mo|ma|mos|mas|to|ta|tos|tas)', '', text) # remove pronouns at the end of the words
    sentences = sent_tokenizer.tokenize(text)
    text = ' '.join([sentence for sentence in sent_tokenizer.tokenize(text) if len(sentence) > 15]) 
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text) # remove punctuation
    text = re.sub('[‘’“”«|»…]', '', text)
    text = re.sub('\w*\d\w*', '', text) # remove words containing numbers
    text = ' '.join([word for word in text.split() if word not in stopwords]) # remove stopwords
    text = ' '.join([word for word in text.split() if len(word) > 2]) # remove words with length <= 2
    text = re.sub(r'Pág.*por pág$', '', text) # remove irrelevant text
    text = ' '.join([word for word in text.split() if word not in irrelevant]) 
    #text = nlp(text)
    #text = ' '.join([word.lemma for sent in text.sentences for word in sent.words]) 

    return text

data_clean = pd.DataFrame(data_text.content.apply(lambda x: clean_text(x)))
data_clean

,content
0,prisão perpétua homem tentou assassinar senadora dia jared loughner tentou matar sucesso então senadora gabrielle giffords eleita partido democrat...
1,john nash matemático mente brilhante morre acidente john nash matemático nobel economia retratado russell crowe filme mente brilhante morreu acide...
2,mito reeleição mínima garantida cavaco sairá desta campanha pior entrou casos acções sln casa férias algarve revelaram qualquer ilegalidade pudess...
3,morreu rita levintalcini grande dama ciência italiana cientista senadora italiana rita levintalcini prémio nobel medicina morreu neste anos casa r...
4,trás porta amarela homem problemas psicológicos ninguém sabe fazer ninguém sabe pensa come sobrevive dois anos desde mulher pânico levou dois filh...
5,gregorio duvivier todas maneiras estilo mês gregorio duvivier portugal escritor humorista actorsócio fundador colectivo porta fundos flavour the m...
6,ângelo sousa viver esperar talvez pintar ângelo nome curioso farta esperar procura saber conversa porto casa tudo parado atelier felicidade triste...
7,sintomas esquizofrenia fernando rosas zurze ferreira amaral sintomas esquizofrenia porta serviço estrangeiros fronteiras lisboa cenário escolhido ...
8,esquizofrenia depressão esquizofrenia depressão longo anos existência hospital conde ferreira palco mudanças institucionais acontecerá igualmente ...


In [8]:
# Word tokenization

def tokenize(text):

    text = text.split()

    return text

data_tokenized = pd.DataFrame(data_clean.content.apply(lambda x: tokenize(x)))
data_tokenized

,content
0,"[prisão, perpétua, homem, tentou, assassinar, senadora, dia, jared, loughner, tentou, matar, sucesso, então, senadora, gabrielle, giffords, eleita..."
1,"[john, nash, matemático, mente, brilhante, morre, acidente, john, nash, matemático, nobel, economia, retratado, russell, crowe, filme, mente, bril..."
2,"[mito, reeleição, mínima, garantida, cavaco, sairá, desta, campanha, pior, entrou, casos, acções, sln, casa, férias, algarve, revelaram, qualquer,..."
3,"[morreu, rita, levintalcini, grande, dama, ciência, italiana, cientista, senadora, italiana, rita, levintalcini, prémio, nobel, medicina, morreu, ..."
4,"[trás, porta, amarela, homem, problemas, psicológicos, ninguém, sabe, fazer, ninguém, sabe, pensa, come, sobrevive, dois, anos, desde, mulher, pân..."
5,"[gregorio, duvivier, todas, maneiras, estilo, mês, gregorio, duvivier, portugal, escritor, humorista, actorsócio, fundador, colectivo, porta, fund..."
6,"[ângelo, sousa, viver, esperar, talvez, pintar, ângelo, nome, curioso, farta, esperar, procura, saber, conversa, porto, casa, tudo, parado, atelie..."
7,"[sintomas, esquizofrenia, fernando, rosas, zurze, ferreira, amaral, sintomas, esquizofrenia, porta, serviço, estrangeiros, fronteiras, lisboa, cen..."
8,"[esquizofrenia, depressão, esquizofrenia, depressão, longo, anos, existência, hospital, conde, ferreira, palco, mudanças, institucionais, acontece..."


In [9]:
# Write to csv

data_tokenized.to_csv("data_cleaned.csv")

In [10]:
# Generate bag-of-words feature vectors

vectorizer = CountVectorizer(lowercase=False)
bag_of_words = vectorizer.fit_transform(data_clean['content'].values)

#print(vectorizer.get_feature_names())
print(bag_of_words.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 1 0 ... 4 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [11]:
# Generate TF-IDF feature vectors

tfidf_vectorizer = TfidfVectorizer()
tfidf = tfidf_vectorizer.fit_transform(data_clean['content'].values)

print(tfidf.toarray())

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.01064605 0.         ... 0.0359673  0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
